# Spend Data

In [ ]:
import requests
import pandas as pd
import sys
from io import BytesIO
import gzip
from tqdm import tqdm
import io
import os
from global_variables import *

In [2]:
page = 1
download_count = 0
PRODUCT_API_PATH = 'https://app.deweydata.io/external-api/v3/products/eb6e748a-0fdd-4bc7-9dd7-bbed0890948d/files'
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/wireless_spending_20210101_20221231.csv'
while True:
    # get results from API endpoint, using API key for authentication
    results = requests.get(url=PRODUCT_API_PATH,
                           params={'page': page,
                                   'partition_key_after': '2021-01-01',   # set date value here
                                   'partition_key_before': '2022-12-31'}, # set date value here
                           headers={'X-API-KEY': API_KEY,
                                    'accept': 'application/json'
                                   })
    response_json = results.json()

    # for each result page, loop through download links and save to your computer
    for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")
    
        data = requests.get(link_data['link'])
        gzip_stream = io.BytesIO(data.content)
    
        df = pd.read_csv(gzip_stream, compression='gzip', low_memory=False)
        filtered_df = df[df['BRANDS'].isin(['T-Mobile','AT&T','Verizon Wireless','Cricket Wireless','Boost Mobile','Wireless Zone','U.S. Cellular'])  ]
        filtered_df = filtered_df[['PLACEKEY','SAFEGRAPH_BRAND_IDS','BRANDS','SPEND_DATE_RANGE_START','RAW_TOTAL_SPEND','RAW_NUM_TRANSACTIONS','RAW_NUM_CUSTOMERS','SPEND_BY_DAY','MEAN_SPEND_PER_CUSTOMER_BY_INCOME','RELATED_PAYMENT_PLATFORM_PCT','RELATED_WIRELESS_CARRIER_PCT']]
        
        if os.path.isfile(ft_data_path):
            header_option = False  
        else:
            header_option = True  
    
        filtered_df.to_csv(ft_data_path, mode='a', header=header_option, index=False)
        
        download_count += 1

    total_pages = response_json['total_pages']
    if page >= total_pages:
        break
    page += 1   

In [5]:
df = pd.read_csv(ft_data_path)
len(df)
df = df.dropna(subset=['RAW_NUM_TRANSACTIONS'], how='any')
df = df.reset_index()
len(df)

154453

In [9]:
import json
temp = df.loc[0]
data = json.loads(temp['SPEND_BY_DAY'])
data = pd.DataFrame({'day_no': range(1, len(data) + 1), 'spend_amount': data})
data['PLACEKEY'] = temp['PLACEKEY']
data['DATE_RANGE_START'] = pd.to_datetime(temp['SPEND_DATE_RANGE_START'])
data


,day_no,spend_amount,PLACEKEY,DATE_RANGE_START
0,1,0.00,22r-222@5xq-cx4-zzz,2021-01-01
1,2,0.00,22r-222@5xq-cx4-zzz,2021-01-01
2,3,0.00,22r-222@5xq-cx4-zzz,2021-01-01
3,4,0.00,22r-222@5xq-cx4-zzz,2021-01-01
4,5,0.00,22r-222@5xq-cx4-zzz,2021-01-01
5,6,55.73,22r-222@5xq-cx4-zzz,2021-01-01
6,7,0.00,22r-222@5xq-cx4-zzz,2021-01-01
7,8,0.00,22r-222@5xq-cx4-zzz,2021-01-01
8,9,0.00,22r-222@5xq-cx4-zzz,2021-01-01
9,10,0.00,22r-222@5xq-cx4-zzz,2021-01-01


In [10]:
import json
header_option = True 
daily_spend_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/wireless_spending_daily_20210101_20221231.csv'

with tqdm(total=len(df)) as pbar:
    
    for index in range(0, len(df)):
        
        if os.path.isfile(daily_spend_file):
            header_option = False  
            
        temp = df.loc[index] 
        data = json.loads(temp['SPEND_BY_DAY'])
        data = pd.DataFrame({'day_no': range(1, len(data) + 1), 'spend_amount': data})
        data['PLACEKEY'] = temp['PLACEKEY']
        data['DATE_RANGE_START'] = pd.to_datetime(temp['SPEND_DATE_RANGE_START'])
        data.to_csv(daily_spend_file, index=False, mode='a', header=header_option,sep="|")

        pbar.update(1)

100%|██████████| 154453/154453 [07:45<00:00, 332.07it/s]


In [11]:
poi_csv_path = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_poi_with_store_id.csv.gz'
poi = pd.read_csv(poi_csv_path) 

In [13]:
wireless = poi[poi['PLACEKEY'].isin(df['PLACEKEY'])]

wireless.to_parquet('C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/wireless_poi.parquet')

In [15]:
df.to_parquet('C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/wireless_spending_summary.parquet')

In [17]:
daily = pd.read_csv(daily_spend_file,sep="|")
daily.to_parquet(daily_spend_file+'.parquet')

# Foot Traffic Data

In [26]:
date_start = '2021-01-01'
date_end = '2022-12-31'

In [27]:
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/wireless_weekly_ft.csv'
page = 1
download_count = 0
while True:
    print(page)
    results = requests.get(url='https://app.deweydata.io/external-api/v3/products/50f21464-9d1b-4f44-89b5-f633ac31e64f/files',
                       params={'page': page,
                               'partition_key_after': date_start,   # optionally set date value here
                               'partition_key_before': date_end}, # optionally set date value here
                       headers={'X-API-KEY': API_KEY,
                                'accept': 'application/json'
                               })
    response_json = results.json()

    for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")
    
        data = requests.get(link_data['link'])
        gzip_stream = io.BytesIO(data.content)
    
        df = pd.read_csv(gzip_stream, compression='gzip', low_memory=False)
        filtered_df = df[df['placekey'].isin(wireless['PLACEKEY'])  ]
        filtered_df = filtered_df[['placekey','location_name','brands','device_type','visitor_home_aggregation','visits_by_day','raw_visitor_counts','raw_visit_counts','date_range_start']]
        
        if os.path.isfile(ft_data_path):
            header_option = False  
        else:
            header_option = True  
    
        filtered_df.to_csv(ft_data_path, mode='a', header=header_option, index=False)
        
        download_count += 1
        
    total_pages = response_json['total_pages']
    if page >= total_pages:
        break
    page += 1   

1
2


KeyboardInterrupt: 

In [78]:
df = pd.read_csv(ft_data_path)
df = df.dropna(subset=['RAW_VISIT_COUNTS'], how='any')
df = df.reset_index()

In [79]:
df[['PLACEKEY','DATE_RANGE_START','RAW_VISIT_COUNTS','RAW_VISITOR_COUNTS','DISTANCE_FROM_HOME','MEDIAN_DWELL']].to_parquet(ft_data_path+"_visits_summary.parquet", index=False)

In [80]:
import json
header_option_visit_summary_file = True 
header_visit_cbgs_file = True
visit_summary_file = ft_data_path+"_visits.csv"
visit_cbgs_file = ft_data_path+"_cbgs.csv"

with tqdm(total=len(df)) as pbar:
    
    for index in range(0, len(df)):
        
        if os.path.isfile(visit_summary_file):
            header_option_visit_summary_file = False  
            
        if os.path.isfile(visit_cbgs_file):
            header_visit_cbgs_file = False  
            
        temp = df.loc[index] 
        
        data = json.loads(temp['VISITS_BY_DAY'])
        data = pd.DataFrame({'day_no': range(1, len(data) + 1), 'no_visits': data})
        data['PLACEKEY'] = temp['PLACEKEY']
        data['DATE_RANGE_START'] = pd.to_datetime(temp['DATE_RANGE_START'])
        data.to_csv(visit_summary_file, index=False, mode='a', header=header_option_visit_summary_file,sep="|")
        
        if str(temp['VISITOR_HOME_CBGS'])!='nan':
            data_loc = json.loads(temp['VISITOR_HOME_CBGS'])
            df1 = pd.DataFrame(data_loc.items(), columns=['CBGS', 'no_visits'])
            df1['PLACEKEY'] = temp['PLACEKEY']
            df1['DATE_RANGE_START'] = temp['DATE_RANGE_START']
            df1.to_csv(visit_cbgs_file, index=False, mode='a', header=header_visit_cbgs_file,sep="|")

        pbar.update(1)

100%|██████████| 393163/393163 [35:41<00:00, 183.63it/s] 


In [81]:
df1 = pd.read_csv(visit_summary_file,sep="|",low_memory=False)  
df1.to_parquet(visit_summary_file+".parquet", index=False)
df2 = pd.read_csv(visit_cbgs_file,sep="|",low_memory=False)
df2.to_parquet(visit_cbgs_file+".parquet", index=False)

# Spending

In [ ]:
page = 1
download_count = 0
PRODUCT_API_PATH = 'https://app.deweydata.io/external-api/v3/products/eb6e748a-0fdd-4bc7-9dd7-bbed0890948d/files'
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/wireless_spending_20210101_20221231.csv'
while True:
    # get results from API endpoint, using API key for authentication
    results = requests.get(url=PRODUCT_API_PATH,
                           params={'page': page,
                                   'partition_key_after': '2021-01-01',   # set date value here
                                   'partition_key_before': '2022-12-31'}, # set date value here
                           headers={'X-API-KEY': API_KEY,
                                    'accept': 'application/json'
                                   })
    response_json = results.json()

    # for each result page, loop through download links and save to your computer
    for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")
    
        data = requests.get(link_data['link'])
        gzip_stream = io.BytesIO(data.content)
    
        df = pd.read_csv(gzip_stream, compression='gzip', low_memory=False)
        filtered_df = df[df['BRANDS'].isin(['T-Mobile','AT&T','Verizon Wireless','Cricket Wireless','Boost Mobile','Wireless Zone','U.S. Cellular'])  ]
        filtered_df = filtered_df[['PLACEKEY','SAFEGRAPH_BRAND_IDS','BRANDS','SPEND_DATE_RANGE_START','RAW_TOTAL_SPEND','RAW_NUM_TRANSACTIONS','RAW_NUM_CUSTOMERS','SPEND_BY_DAY','MEAN_SPEND_PER_CUSTOMER_BY_INCOME','RELATED_PAYMENT_PLATFORM_PCT','RELATED_WIRELESS_CARRIER_PCT']]
        
        if os.path.isfile(ft_data_path):
            header_option = False  
        else:
            header_option = True  
    
        filtered_df.to_csv(ft_data_path, mode='a', header=header_option, index=False)
        
        download_count += 1

    total_pages = response_json['total_pages']
    if page >= total_pages:
        break
    page += 1   


In [99]:
df = pd.read_csv(ft_data_path)
df = df.dropna(subset=['RAW_TOTAL_SPEND'], how='any')
df = df.reset_index()
df.head()

,index,PLACEKEY,SAFEGRAPH_BRAND_IDS,BRANDS,SPEND_DATE_RANGE_START,RAW_TOTAL_SPEND,RAW_NUM_TRANSACTIONS,RAW_NUM_CUSTOMERS,SPEND_BY_DAY,MEAN_SPEND_PER_CUSTOMER_BY_INCOME,RELATED_PAYMENT_PLATFORM_PCT,RELATED_WIRELESS_CARRIER_PCT
0,0,22r-222@5xq-cx4-zzz,SG_BRAND_5deb800ce9500e72e355137ab8b48fb6,AT&T,2021-01-01 00:00:00.000,138.85,3,3,"[0.00,0.00,0.00,0.00,0.00,55.73,0.00,0.00,0.00...","{""45-60k"":52.50,""60-75k"":55.73,""75-100k"":30.62}",{},"{""Sprint"":34,""Verizon Wireless"":34}"
1,1,22s-222@627-s6m-yqf,SG_BRAND_4b82356db1a8f4a2db26dd5b7e30abba,T-Mobile,2021-01-01 00:00:00.000,5582.96,46,33,"[0.00,0.00,0.00,662.29,718.82,162.93,13.44,97....","{""<25k"":280.56,""25-45k"":173.27,""45-60k"":184.49...","{""Cash App"":4}","{""Sprint"":4,""AT&T"":4,""Disney Mobile"":4}"
2,2,zzy-223@5x4-2jm-p7q,SG_BRAND_4b82356db1a8f4a2db26dd5b7e30abba,T-Mobile,2021-01-01 00:00:00.000,1801.23,16,13,"[0.00,0.00,0.00,0.00,59.98,119.16,0.00,167.73,...","{""<25k"":42.87,""25-45k"":102.56,""45-60k"":114.21,...","{""PayPal"":8,""Apple Card"":8,""Apple Cash"":8}","{""Sprint"":16,""AT&T"":8}"
3,3,zzw-222@5z6-3r6-7h5,SG_BRAND_4b82356db1a8f4a2db26dd5b7e30abba,T-Mobile,2021-01-01 00:00:00.000,1882.38,11,8,"[0.00,0.00,33.49,325.23,0.00,232.78,0.00,0.00,...","{""<25k"":134.73,""25-45k"":125.93,""75-100k"":993.64}",{},{}
4,4,zzy-224@63r-gs7-s3q,SG_BRAND_4b82356db1a8f4a2db26dd5b7e30abba,T-Mobile,2021-01-01 00:00:00.000,1587.00,16,9,"[0.00,0.00,0.00,483.25,0.00,0.00,0.00,324.71,0...","{""<25k"":148.80,""25-45k"":206.14,""45-60k"":236.88...","{""PayPal"":12}","{""AT&T"":23,""Sprint"":12,""Verizon Wireless"":12}"


In [102]:
brand_counts = df['BRANDS'].value_counts()
brand_counts.head()

BRANDS
T-Mobile            64204
AT&T                44404
Verizon Wireless    20720
Cricket Wireless    14370
Boost Mobile         4330
Name: count, dtype: int64